# Import packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib.ticker as ticker
import time as time
import itertools
import scipy as sp
import scipy.misc
from scipy import stats
from scipy.interpolate import interp1d
import nbimporter
import csv

# $f_\psi$ for elliptical and circular orbits with $2$ visible planets

In [2]:
def Tperiod(Mstar, a): # period for elliptical orbit
    'Mstar- mass of the star, a-semi-major axis'
    G = 1 # units of the gravitational constant
    return 2*np.pi * np.sqrt(a**3/(G*Mstar)) # formula

In [3]:
def projCoor(i, O, w, x, y): # Solar System Dynamics, Murray, page 50(64)
    P1 = np.array([[np.cos(w), -np.sin(w), 0],[np.sin(w), np.cos(w), 0],[0, 0, 1]])
    P2 = np.array([[1, 0, 0],[0, np.cos(i), -np.sin(i)],[0, np.sin(i), np.cos(i)]])
    P3 = np.array([[np.cos(O), -np.sin(O), 0],[np.sin(O), np.cos(O), 0],[0, 0, 1]])
    return P3@P2@P1@np.array([x,y,np.zeros(len(x))])

In [4]:
def NewtonRTol(e, M, tol=10**-4): # Newton-Raphson method to find roots
    'e-eccentricity, M-mean anomaly, tol-tolerance'
    k = .85 # by Danby (1988)
    E0 = M + np.sign(np.sin(M))*k*e
    fE0 = E0 - e*np.sin(E0) - M
    while tol<abs(fE0):
        E0 = E0 - fE0/(1-e*np.cos(E0))
        fE0 = E0 - e*np.sin(E0) - M
    return E0

*PsiElli* has an output $f_\psi$ for any given system with $2$ planets. It is used in sections **2. Circular orbit** and **3. Elliptical orbit**. 

In [5]:
def PsiElli(Mstar, a1, e1, i1, O1, w1, a2, e2, i2, O2, w2):
    ## Exceptions for invalid values of the input
    if Mstar<=0 or a1<=0 or a2<=0:
        raise ValueError('The stars mass and the radius of the planets are positive quantities.')
    if not (0<=i1<=np.pi/2) or not (0<=i2<=np.pi/2):
        raise ValueError('The inclination of the orbit is between 0 and pi/2.') 
    if not (0<=O1<=2*np.pi) or not (0<=O2<=2*np.pi):
        raise ValueError('The longitude of the ascending node of the orbit and the true anomaly at M0 are between 0 and 2*pi.')
    if a1==a2 and i1==i2 and O1==O2:
        raise ValueError('The planets can not have the same parameters, aka being the same.') 
    if e1>=1 or e1<0 or e2>=1 or e2<0:
        raise ValueError('An elipstic orbit have eccentricity between 0 and 1 (0<e<1).')
    
    Ntimes = 10**7 # # of 'photos' to take of the system
    Nbins = 10**2 # # of bins in the Psi histogram
    fonty = 18 # fontsize
    
    T1, T2 = Tperiod(Mstar, a1), Tperiod(Mstar, a2) # period of each of the planets
    n1, n2 = 2*np.pi/T1, 2*np.pi/T2 # mean motion of each planet
    ts1 = T1 * np.random.random_sample(Ntimes) # Ntimes random times for making the Psi graph
    ts2 = T2 * np.random.random_sample(Ntimes) # Ntimes random times for making the Psi graph
    
    Ms1, Ms2 = n1*ts1, n2*ts2 # mean anomaly of each planet (I assume t_periapse=0 because it will not matter)
    E1, E2 = np.zeros(Ntimes), np.zeros(Ntimes)
    for i in np.arange(Ntimes):
        E1[i] = NewtonRTol(e1, Ms1[i])
        E2[i] = NewtonRTol(e2, Ms2[i])
    
    xs1, ys1 = a1*(np.cos(E1)-e1), a1*np.sqrt(1-e1**2)*np.sin(E1)
    xs2, ys2 = a2*(np.cos(E2)-e2), a2*np.sqrt(1-e2**2)*np.sin(E2)
    XYZ1 = projCoor(i1, O1, w1, xs1, ys1)
    XYZ2 = projCoor(i2, O2, w2, xs2, ys2)
    cosPsi = np.sum(XYZ1[:2]*XYZ2[:2], axis=0) / (np.linalg.norm(XYZ1[:2], axis=0) * np.linalg.norm(XYZ2[:2], axis=0))
    cosPsi[cosPsi>1], cosPsi[cosPsi<-1] = 1, -1 # numeric aproximations
    PsiVals = np.arccos(cosPsi) # angle between the planets
    
    # Plotting a graph
    plt.ylabel("$f_{\psi}$", fontsize=fonty)
    PsiVals = PsiVals*180/np.pi
    xPretty = np.arange(0, 181, 30)
    plt.xticks(xPretty)
    plt.xlabel("$\psi$ (\N{DEGREE SIGN})", fontsize=fonty)
    y, binEdges = np.histogram(PsiVals, bins=Nbins, density=True)
    bincenters = 0.5*(binEdges[1:]+binEdges[:-1])
    plt.ylim(0, np.max(y)*1.2)
    plt.plot(bincenters, y, '-')
    plt.tick_params(axis='both', which='major', labelsize=fonty-2)
    return np.max(y)

# $f_{\psi_{min}}$ and $f_{\psi_{max}}$ for $N$ visible planets with elliptical orbits in a given system

I used this code on section **4. Multi-planet systems**.

In [6]:
def PsiN(ess, iss, Oss, wss, scatter=0, plot=(1,1), col=0, lin='-'):
    ## Exceptions for invalid values of the input
    if len(ess) != len(iss) or len(ess) != len(Oss) or len(ess) != len(wss):
        raise ValueError('We need to have an e, i, O and w for each planet.')
    if np.any(iss<0) or np.any(iss>np.pi/2): 
        raise ValueError('The inclination of the orbit is between 0 and pi/2.') 
    if np.any(Oss<0) or np.any(Oss>2*np.pi): 
        raise ValueError('The longitude of the ascending node of the orbit and the true anomaly at M0 are between 0 and 2*pi.')
    if np.any(ess<0) or np.any(ess>=1): 
        raise ValueError('An elipstic orbit have eccentricity between 0 and 1 (0<e<1).')
    
    # Fixed hings I might want to change
    Ntimes = 10**7 # # of 'photos' to take of the system
    Nbins = 10**2 # # of bins in the Psi histogram
    fonty = 26 # fontsize
    
    Mstar = 1 # it is irrelevant, so I will just set to one
    Nplanets = len(ess) # # planets is the number of different eccentricities we have
    ass = np.reshape(np.ones(Nplanets), (Nplanets, 1)) # it is irrelevant so I will set to 1. 
                                    # However, if they are all in the same orbit, the code poduces wrong result
    ess = np.reshape(ess, (Nplanets, 1))
    
    Ts = np.reshape(Tperiod(Mstar, ass), (Nplanets,1)) # period of each of the planets
    ns = 2*np.pi/Ts # mean motion of each planet
    tss = np.multiply(Ts, np.random.random_sample((Nplanets, Ntimes)))
    
    Mss = ns*tss # mean anomaly of each planet (I assume t_periapse=0 because it will not matter)
    Ess = np.zeros((Nplanets, Ntimes))
    for i in np.arange(Ntimes):
        for j in np.arange(Nplanets):
            Ess[j,i] = NewtonRTol(ess[j], Mss[j,i])
    xs = ass * (np.cos(Ess)-ess)
    ys = ass*np.sqrt(1-ess**2)*np.sin(Ess)
    
    Psis, j = np.zeros((int(sp.special.comb(Nplanets, 2)), Ntimes)), 0
    PossibleNs = np.arange(Nplanets)
    for i in itertools.combinations(PossibleNs, 2):
        XYZ1 = projCoor(iss[i[0]], Oss[i[0]], wss[i[0]], xs[i[0]], ys[i[0]])
        XYZ2 = projCoor(iss[i[1]], Oss[i[1]], wss[i[1]], xs[i[1]], ys[i[1]])
        cosPsi = np.sum(XYZ1[:2]*XYZ2[:2], axis=0) / (np.linalg.norm(XYZ1[:2], axis=0) * np.linalg.norm(XYZ2[:2], axis=0))
        if np.sum(cosPsi>1)+np.sum(cosPsi<-1) != 0:
            print(np.sum(cosPsi>1)+np.sum(cosPsi<-1)) # see that it is doing a numeric aproximation
            cosPsi[cosPsi>1], cosPsi[cosPsi<-1] = 1, -1 # numeric aproximations
        Psis[j] = np.arccos(cosPsi) # angle between the planets
        j += 1
    
    minPsi = np.min(Psis, axis=0)
    maxPsi = np.max(Psis, axis=0)
    meanPsi = np.mean(Psis, axis=0)
    if deg != ():
        minPsi, maxPsi = minPsi*180/np.pi, maxPsi*180/np.pi
        plt.xticks(np.arange(0, 181, 30))
    
    if scatter==1: # DO a scatter plot (figure 4.2)
        if Ntimes <= 10**6:
            plt.xlabel("$\psi_{min}$", fontsize=fonty)
            plt.ylabel("$\psi_{max}$", fontsize=fonty)
            plt.yticks(np.arange(0, 181, 30))
            plt.xticks(np.arange(0, 121, 30))
            plt.plot(minPsi, maxPsi,'o', markersize=1)
            plt.tick_params(axis='both', which='major', labelsize=fonty-2)
        else:
            print('The Scatter plot is not presented because the sample size is too big. 10**4 or 10**5 is good.')
    
    if np.sum(plot) != 0: # we want to plot some psi distributions
        plt.xlabel("$\psi$", fontsize=fonty)
        plt.ylabel("$f_{\psi_{min}}$", fontsize=fonty)
        maxi = 0
        if plot[0] == 1: # plot the minimum angle distribution
            y, binEdges = np.histogram(minPsi, bins=Nbins, density=True)
            bincenters = 0.5*(binEdges[1:]+binEdges[:-1])
            plt.plot(bincenters, y, color=col, ls=lin, label='$\psi_{min}$')
            maxi = np.max(y) # maximum value of the density
        if plot[1] == 1: # plot the maximum angle distribution
            y, binEdges = np.histogram(maxPsi, bins=Nbins, density=True)
            bincenters = 0.5*(binEdges[1:]+binEdges[:-1])
            if plot[0] == 1:
                plt.plot(bincenters, y, color=col, ls='--', label='$\psi_{max}$')
            else:                
                plt.plot(bincenters, y, color=col, ls=lin, label='$\psi_{max}$')
            maxi = max(maxi, np.max(y)) # maximum value of the density
        plt.ylim(0, maxi*1.2)
        plt.tick_params(axis='both', which='major', labelsize=fonty-2)
        plt.xlabel("$\psi_{min}$ (\N{DEGREE SIGN})", fontsize=fonty)
        if plot[1] == 1:
            plt.xlabel("$\psi_{max}$ (\N{DEGREE SIGN})", fontsize=fonty)
            if plot[0] == 1:
                plt.xlabel("$\psi$ (\N{DEGREE SIGN})", fontsize=fonty)
        return maxi

# $f_{\psi_{min}}$ and $f_{\psi_{max}}$ for $N$ visible planets in a population 
I used this code on section **5. Distributions**.

In [7]:
def fPsiStar_Plot(planar, Nstar, plot=(1,0)):
    'Nstar - # of planets orbiting the stars'
    Npics = 10**6 # # of systems to whoom we take pics
    Nbins = 10**2 # # of bins in the histogram
    fonty = 26 # fontsize
    
    # Depend on
    Os = np.pi*np.random.random_sample((Nstar, Npics))
    iis = np.arccos(np.random.random_sample((Nstar, Npics))) # random inclinations for each planet
    ws = 2*np.pi*np.random.random_sample((Nstar, Npics)) # random ws for each system
    es = np.random.random_sample((Nstar, Npics)) # random eccentricities for each system
    
    lin = '--'
    if planar == 1: # I want to simulate coplanar systems
        iis = np.zeros((Nstar, Npics)) + iis[0]
        Os = np.zeros((Nstar, Npics)) + Os[0]
        lin = '-'
        
    Mstar = 1
    a = 1 # independent of a distribution
    T = Tperiod(Mstar, a)
    n = 2*np.pi/T # mean motion of each planet
    tss = T * np.random.random_sample((Nstar, Npics)) # I will not use all the times, but that is fine
    Mss = n*tss # mean anomaly of each planet (I assume t_periapse=0 because it will not matter)
    Ess = np.zeros((Nstar, Npics)) - 666 # just to be sure to get a weird result outside the limits
    
    minMaxPsi = np.zeros((2, Npics))
    for i in np.arange(Npics): # Calculate the angles for each system
        Es = np.zeros(Nstar)
        XYZs = np.zeros((3, Nstar))
        for j in np.arange(Nstar): # Calculate a position for each planet orbiting the star
            E = NewtonRTol(es[j,i], Mss[j,i])
            x, y = a*(np.cos(E)-es[j,i]), a*np.sqrt(1-es[j,i]**2)*np.sin(E)
            XYZs[:,j] = projCoor(iis[j,i], Os[j,i], ws[j,i], [x], [y]).T
        Psis, j = np.zeros(int(sp.special.comb(Nstar, 2))), 0
        PossibleNs = np.arange(Nstar)
        for k in itertools.combinations(PossibleNs, 2): # Calculate all angles between the planets
            XYZ1 = XYZs[:,k[0]]
            XYZ2 = XYZs[:,k[1]]
            cosPsi = np.sum(XYZ1[:2]*XYZ2[:2], axis=0) / (np.linalg.norm(XYZ1[:2], axis=0) * np.linalg.norm(XYZ2[:2], axis=0))
            if cosPsi>1:
                print(cosPsi) # see that it is doing a numeric aproximation
                cosPsi = 1 # numeric aproximation
            elif cosPsi < -1:
                print(cosPsi) # see that it is doing a numeric aproximation
                cosPsi = -1 # numeric aproximation
            Psis[j] = np.arccos(cosPsi) # angle between the planets
            j +=1
        minMaxPsi[:,i] = np.array([np.min(Psis), np.max(Psis)]).T
        
    minMaxPsi = minMaxPsi*180/np.pi
    plt.xticks(np.arange(0, 181, 30))
    maxi = 0
    if plot[0] == 1: # plot the minimum angle distribution
        y, binEdges = np.histogram(minMaxPsi[0], bins=Nbins, density=True)
        bincenters = 0.5*(binEdges[1:]+binEdges[:-1])
        plt.plot(bincenters, y, label='$\psi_{min}$', linestyle=lin, color='C'+str(Nstar-2))
        plt.ylabel("$f_{\psi_{min}}$", fontsize=fonty) # 14-normal; 18-4 fig
        maxi = np.max(y) # maximum value of the density
    if plot[1] == 1: # plot the maximum angle distribution
        y, binEdges = np.histogram(minMaxPsi[1], bins=Nbins, density=True)
        bincenters = 0.5*(binEdges[1:]+binEdges[:-1])
        plt.plot(bincenters, y, label='$\psi_{max}$', linestyle=lin, color='C'+str(Nstar-2))
        plt.ylabel("$f_{\psi_{max}}$", fontsize=fonty) # 14-normal; 18-4 fig
        maxi = max(maxi, np.max(y)) # maximum value of the density
    if np.sum(plot) == 2:
        plt.ylabel("$f_{\psi}$", fontsize=fonty) # 14-normal; 18-4 fig
    plt.ylim(0, maxi*1.2)
    plt.tick_params(axis='both', which='major', labelsize=fonty-2)
    plt.xlabel("$\psi$ (\N{DEGREE SIGN})", fontsize=fonty)
    return maxi

# Codes for hypothesis testing

They were used in **6. Testing coplanarity** and **7. HR 8799**.

*fPsiStar* is the same as *fPsiStar_Plot* but instead of returning graphs, it returns The y and x coordinates to plot $f\psi$ and the values used to construct the histogram that originated $f_{\psi}$.

In [8]:
def fPsiStar(planar, Nstar, getf=(1,0), val=0):
    if Nstar<2:
        raise ValueError('There must be at least 2 stars.')
    if np.sum(getf)<=0:
        raise ValueError('We need to want to get at least one distribution.')
    Npics = 10**7 # # of systems to whom we take pics
    Nbins = 10**2 # # of bins in the histogram
    
    Os = np.pi*np.random.random_sample((Nstar, Npics))
    iis = np.arccos(np.random.random_sample((Nstar, Npics))) # random inclinations for each planet 
    ws = 2*np.pi*np.random.random_sample((Nstar, Npics)) # random ws for each system
    es = np.random.random_sample((Nstar, Npics)) # random eccentricities for each system
    
    lin = '--'
    if planar == 1:
        iis = np.zeros((Nstar, Npics)) + iis[0]
        Os = np.zeros((Nstar, Npics)) + Os[0]
        lin = '-'
        
    Mstar, a = 1, 1 # independent of a distribution
    T = Tperiod(Mstar, a)
    n = 2*np.pi/T # mean motion of each planet
    tss = T * np.random.random_sample((Nstar, Npics)) # I will not use all the times, but that is fine
    Mss = n*tss # mean anomaly of each planet (I assume t_periapse=0 because it will not matter)
    
    minMaxPsi = np.zeros((2, Npics))
    for i in np.arange(Npics):
        XYZs = np.zeros((3, Nstar))
        for j in np.arange(Nstar):
            E = NewtonRTol(es[j,i], Mss[j,i])
            x, y = a*(np.cos(E)-es[j,i]), a*np.sqrt(1-es[j,i]**2)*np.sin(E)
            XYZs[:,j] = projCoor(iis[j,i], Os[j,i], ws[j,i], [x], [y]).T
        Psis, j = np.zeros(int(sp.special.comb(Nstar, 2))), 0
        PossibleNs = np.arange(Nstar)
        for k in itertools.combinations(PossibleNs, 2):
            XYZ1, XYZ2 = XYZs[:,k[0]], XYZs[:,k[1]]
            cosPsi = np.sum(XYZ1[:2]*XYZ2[:2], axis=0) / (np.linalg.norm(XYZ1[:2], axis=0) * np.linalg.norm(XYZ2[:2], axis=0))
            if np.abs(cosPsi)>1:
                print(cosPsi)
                cosPsi = np.sign(cosPsi)
            Psis[j] = np.arccos(cosPsi) # angle between the planets
            j +=1
        minMaxPsi[:,i] = np.array([np.min(Psis), np.max(Psis)]).T
    
    if getf[0] == 1: # minimum angle distribution
        yMin, binEdges = np.histogram(minMaxPsi[0], bins=Nbins, density=True)
        bincentersMin = 0.5*(binEdges[1:]+binEdges[:-1])
        if np.sum(getf) == 1:
            if val == 1:
                return bincentersMin, yMin, minMaxPsi[0] # f_psi(PSI), PSI, psi values
            return bincentersMin, yMin
    if getf[1] == 1:
        yMax, binEdges = np.histogram(minMaxPsi[1], bins=Nbins, density=True)
        bincentersMax = 0.5*(binEdges[1:]+binEdges[:-1])
        if np.sum(getf) == 1:
            if val == 1:
                return bincentersMax, yMax, minMaxPsi[1]
            return bincentersMax, yMax
        else:
            if val == 1:
                return bincentersMin, yMin, minMaxPsi[0], bincentersMax, yMax, minMaxPsi[1]
            return bincentersMin, yMin, bincentersMax, yMax

*PsiVals* is similar to *fPsiStar* but from it we can only obtain samples of $f_{\psi}$. 

In [9]:
def PsiVals(planar, Nstar, Nvals, getPsi=(1,0)):
    if Nstar<2:
        raise ValueError('There must be at least 2 stars.')
    if np.sum(getPsi)<=0:
        raise ValueError('We need to want to get PsiMin, PsiMax or both.')
    Os = np.pi*np.random.random_sample((Nstar, Nvals))
    iis = np.arccos(np.random.random_sample((Nstar, Nvals))) # random inclinations for each planet 
    ws = 2*np.pi*np.random.random_sample((Nstar, Nvals)) # random ws for each system
    es = np.random.random_sample((Nstar, Nvals)) # random eccentricities for each system
    
    lin = '--'
    if planar == 1:
        iis = np.zeros((Nstar, Nvals)) + iis[0]
        Os = np.zeros((Nstar, Nvals)) + Os[0]
        lin = '-'
        
    Mstar, a = 1, 1 # independent of a distribution
    T = Tperiod(Mstar, a)
    n = 2*np.pi/T # mean motion of each planet
    tss = T * np.random.random_sample((Nstar, Nvals)) # I will not use all the times, but that is fine
    Mss = n*tss # mean anomaly of each planet (I assume t_periapse=0 because it will not matter)
    
    minMaxPsi = np.zeros((2, Nvals))
    for i in np.arange(Nvals):
        XYZs = np.zeros((3, Nstar))
        for j in np.arange(Nstar):
            E = NewtonRTol(es[j,i], Mss[j,i])
            x, y = a*(np.cos(E)-es[j,i]), a*np.sqrt(1-es[j,i]**2)*np.sin(E)
            XYZs[:,j] = projCoor(iis[j,i], Os[j,i], ws[j,i], [x], [y]).T
        Psis, j = np.zeros(int(sp.special.comb(Nstar, 2))), 0
        PossibleNs = np.arange(Nstar)
        for k in itertools.combinations(PossibleNs, 2):
            XYZ1, XYZ2 = XYZs[:,k[0]], XYZs[:,k[1]]
            cosPsi = np.sum(XYZ1[:2]*XYZ2[:2], axis=0) / (np.linalg.norm(XYZ1[:2], axis=0) * np.linalg.norm(XYZ2[:2], axis=0))
            if np.abs(cosPsi)>1:
                print(cosPsi)
                cosPsi = np.sign(cosPsi)
            Psis[j] = np.arccos(cosPsi) # angle between the planets
            j +=1
        minMaxPsi[:,i] = np.array([np.min(Psis), np.max(Psis)]).T
    
    if getPsi[0] == 1: # plot the minimum angle distribution
        PsiMin = minMaxPsi[0]
        if np.sum(getPsi) == 1:
            return PsiMin
    if getPsi[1] == 1: # plot the maximum angle distribution
        if np.sum(getPsi) == 1:
            return minMaxPsi[1]
        else:
            return PsiMin, minMaxPsi[1]

### Interpolate the value of $f_{\psi}(\psi^\star)$ for a given $\psi^\star$.

In [10]:
def fPsi(PsisNew, PsisN, fPsisN, loglike=0): # in order to calculate the 'right' value of fPsi for the given Psis
    fNew = np.zeros_like(PsisNew)
    full = PsisN[2] - PsisN[1]
    for i in np.arange(len(PsisNew)):
        index = np.sum(PsisN < PsisNew[i])
        if index == 0:
            fNew[i] = fPsisN[index]
        elif index == len(fPsisN):
            fNew[i] = fPsisN[-1]
        else:
            fNew[i] = ( fPsisN[index-1]*(PsisN[index]-PsisNew[i]) + fPsisN[index]*(PsisNew[i]-PsisN[index-1]) )/full
    if loglike == 1:
        return np.sum(np.log(fNew))
    return fNew

### Calculate the Bayes factor.

In [11]:
def BayesFac(data, PsisH0, fPsisH0, PsisH1, fPsisH1): # the null hypothesis is that they are co-planar
    logLH0 = fPsi(data, PsisH0, fPsisH0, loglike=1) # work with logarithms to reduce numerical errors
    logLH1 = fPsi(data, PsisH1, fPsisH1, loglike=1)
    return np.exp(logLH0-logLH1) # exponentiate to get the Bayes factor from the log(B)

### Calculate the Kullback–Leibler divergence.

In [12]:
def D_KL(PsisP, fPsisP, PsisQ, fPsisQ):
    'Kullback–Leibler divergence from Q to P: D_KL(P||Q)'
    Npoints=10**6 # # of points to integrate
    fP = interp1d(PsisP, fPsisP)
    fQ = interp1d(PsisQ, fPsisQ)
    NewPsis = np.linspace(np.max((np.min(PsisP),np.min(PsisQ))), np.min((np.max(PsisP),np.max(PsisQ))), Npoints)
    intP = np.sum(fP(NewPsis)*(NewPsis[1]-NewPsis[0]))
    intQ = np.sum(fQ(NewPsis)*(NewPsis[1]-NewPsis[0]))
    return -np.sum(fP(NewPsis)/intP*np.log((fQ(NewPsis)/intQ)/(fP(NewPsis)/intP))*(NewPsis[1]-NewPsis[0]))

### Calculate the p-value of a specific system.

In [13]:
def pvalInd(NewPsi, PsiVals, MinMax=(1,0)):
    Nvals = len(PsiVals)
    ps = np.zeros_like(NewPsi)
    if np.sum(MinMax)==2: # N=2 (it is a symmetric function)
        for i in range(len(NewPsi)):
            A = np.sum(PsiVals<=NewPsi[i])
            B = np.sum(PsiVals<=np.pi-NewPsi[i])
            ps[i] = np.abs(A-B)/Nvals
        return ps
    elif MinMax[0]==1: # this is PsiMin (it is a decreasing function)
        for i in range(len(NewPsi)):
            ps[i] = np.sum(NewPsi[i]<=PsiVals)/Nvals
        return ps
    else: # this is PsiMax (it is an increasing function)
        for i in range(len(NewPsi)):
            ps[i] = np.sum(NewPsi[i]>=PsiVals)/Nvals
        return ps

### Calculate the overall p-value of several observations.

In [14]:
def pvalGeral(Nstar, NvalsMax, valPsis, planar=(1,0), MinMax=(1,0)):
    Nrep = 500 # # different samples to calculate the mean
    step = 20
    fonty = 26
    
    Nvals = np.arange(1, NvalsMax+1, step=step) # all the values we are going to use  
    pvals_plan, pvals_gen = np.zeros_like(Nvals, dtype=float), np.zeros_like(Nvals, dtype=float) # of X^2 
    
    if planar[0] == 1: # p-value for co-planar systems
        for j in np.arange(Nrep): # repeat to get a mean p-value
            NewMin_plan_j, NewMax_plan_j = PsiVals(planar=1, Nstar=Nstar, Nvals=NvalsMax, getPsi=(1,1))
            if MinMax[0] == 1 and Nstar!=2:
                ps_plan = pvalInd(NewMin_plan_j, valPsis, MinMax=(1,0))
            elif MinMax[1] == 1 and Nstar!=2:
                ps_plan = pvalInd(NewMax_plan_j, valPsis, MinMax=(0,1))
            else: # N=2
                ps_plan = pvalInd(NewMax_plan_j, valPsis, MinMax=(1,1))
            for i in range(len(Nvals)): # consider only the first Nvals[i] systems as observation
                X2i = - 2*np.sum(np.log(ps_plan[:Nvals[i]])) # Fisher method
                pvals_plan[i] += (1-stats.chi2.cdf(X2i, 2*Nvals[i]))/Nrep
    if planar[1] == 1: # p-value for general systems   
        for j in np.arange(Nrep):  # repeat to get a mean p-value
            NewMin_gen_j, NewMax_gen_j = PsiVals(planar=0, Nstar=Nstar, Nvals=NvalsMax, getPsi=(1,1))
            if MinMax[0] == 1 and Nstar!=2:
                ps_gen = pvalInd(NewMin_gen_j, valPsis, MinMax=(1,0))
            elif MinMax[1] == 1 and Nstar!=2:
                ps_gen = pvalInd(NewMax_gen_j, valPsis, MinMax=(1,0))
            else: # N=2
                ps_gen = pvalInd(NewMax_gen_j, valPsis, MinMax=(1,1)) # min or max is the same
            for i in range(len(Nvals)): # consider only the first Nvals[i] systems as observation
                X2i = - 2*np.sum(np.log(ps_gen[:Nvals[i]])) # Fisher method
                pvals_gen[i] += (1-stats.chi2.cdf(X2i, 2*Nvals[i]))/Nrep  
                
    ps = np.array([pvals_plan, pvals_gen])
    lins = ['-', '--']
    pshow, maxi = [], 0
    for i in np.arange(len(ps)):
        if np.sum(ps[i])!= 0:
            plt.plot(Nvals, ps[i], color='C'+str(Nstar-2), linestyle=lins[i])
            pshow.append(ps[i])
            maxi = max(maxi, max(ps[i]))
        plt.plot(Nvals, 0.05+np.zeros_like(Nvals), color='gray', ls='--')
        plt.ylabel('p-value', fontsize=fonty)
        plt.xlabel('Nvals', fontsize=fonty)
        plt.tick_params(axis='both', which='major', labelsize=fonty-2)
    return Nvals, pshow

# Codes for inclination inference

They were used in **7. HR 8799**.

### *fPsi_i* is similar to *fPsiStar* but for a fixed value of inclination.

**Input:** Number of planets in the system, inclination $i$ of those planets (coplanar case), if we want $f_{\psi_{min}}$ and/or $f_{\psi_{max}}$ for that given $i$

**Output:** $\psi$ points, $f_{\psi}$ ($f_{\psi_{min}}$ and/or $f_{\psi_{max}}$) calculated at those points

In [15]:
def fPsi_i(Nstar, ii, getf=(1,0)): # this is for planar systems
    if Nstar<2:
        raise ValueError('There must be at least 2 planets.')
    if np.sum(getf)<=0:
        raise ValueError('We need to want to get at least one distribution.')
    Npics = 10**6 # # of systems to whom we take pics
    Nbins = 10**2 # # of bins in the histogram
    
    Os = np.zeros((Nstar, Npics))
    iis = np.zeros((Nstar, Npics)) + ii
    ws = 2*np.pi*np.random.random_sample((Nstar, Npics)) # random ws for each system
    es = np.random.random_sample((Nstar, Npics)) # random eccentricities for each system
    
    lin = '-'
    Mstar, a = 1, 1 # independent of a distribution
    T = Tperiod(Mstar, a)
    n = 2*np.pi/T # mean motion of each planet
    tss = T * np.random.random_sample((Nstar, Npics)) # I will not use all the times, but that is fine
    Mss = n*tss # mean anomaly of each planet (I assume t_periapse=0 because it will not matter)
    
    minMaxPsi = np.zeros((2, Npics))
    for i in np.arange(Npics):
        XYZs = np.zeros((3, Nstar))
        for j in np.arange(Nstar):
            E = NewtonRTol(es[j,i], Mss[j,i])
            x, y = a*(np.cos(E)-es[j,i]), a*np.sqrt(1-es[j,i]**2)*np.sin(E)
            XYZs[:,j] = projCoor(iis[j,i], Os[j,i], ws[j,i], [x], [y]).T
        Psis, j = np.zeros(int(sp.special.comb(Nstar, 2))), 0
        PossibleNs = np.arange(Nstar)
        for k in itertools.combinations(PossibleNs, 2):
            XYZ1, XYZ2 = XYZs[:,k[0]], XYZs[:,k[1]]
            cosPsi = np.sum(XYZ1[:2]*XYZ2[:2], axis=0) / (np.linalg.norm(XYZ1[:2], axis=0) * np.linalg.norm(XYZ2[:2], axis=0))
            if np.abs(cosPsi)>1:
                print(cosPsi)
                cosPsi = np.sign(cosPsi)
            Psis[j] = np.arccos(cosPsi) # angle between the planets
            j +=1
        minMaxPsi[:,i] = np.array([np.min(Psis), np.max(Psis)]).T
    
    if getf[0] == 1: # plot the minimum angle distribution
        yMin, binEdges = np.histogram(minMaxPsi[0], bins=Nbins, density=True)
        bincentersMin = 0.5*(binEdges[1:]+binEdges[:-1])
        if np.sum(getf) == 1:
            return bincentersMin, yMin
    if getf[1] == 1: # plot the maximum angle distribution
        yMax, binEdges = np.histogram(minMaxPsi[1], bins=Nbins, density=True)
        bincentersMax = 0.5*(binEdges[1:]+binEdges[:-1])
        if np.sum(getf) == 1:
            return bincentersMax, yMax
        else:
            return bincentersMin, yMin, bincentersMax, yMax

### Calculate the likelihood.

**Input:** Value of $\psi$ for which we want to calculate the likelihood (it is the value we observe in the real images), Number of planets in the system, working with $f_{\psi_{min}}$ or $f_{\psi_{max}}$

**Output:** Inclination points, likelihood of $\psi$ measured at those inclination points

In [16]:
def fPsiSpecific_i(PsiSpecific, Nstar, getf=(1,0)):
    if Nstar<2:
        raise ValueError('There must be at least 2 planets.')
    if np.sum(getf)<=0:
        raise ValueError('We need to want to get at least one distribution.')
    Nvals = 10**2
    
    iis = np.linspace(0, np.pi/2, num=Nvals)
    fPsiVals = np.zeros(Nvals)
    for j in np.arange(Nvals): # calculate f_psi for each i value
        PsisN, fPsisN = fPsi_i(Nstar, iis[j], getf=getf)
        fPsiVals[j] = fPsi(np.array([PsiSpecific]), PsisN, fPsisN, loglike=0) 
    fPsiVals = fPsiVals/np.sum((iis[1]-iis[0])*fPsiVals)
    plt.plot(iis, fPsiVals)
    return iis, fPsiVals

### Calculate the posterior distribution given the likelihood.

**Input:** Inclination points, likelihood of $\psi$ measured at those inclination points

**Output:** The normalized posterior at the given inclination points

In [17]:
def posterior(xlike, like):
    seno = np.sin(xlike) # prior i distribution
    post = seno * like # posterior proportional to prior * likelihood
    norm = np.sum(post)*(xlike[1]-xlike[0]) 
    return post/norm # normalize posterior distribution